## Импорт библиотек

Загружаются основные библиотеки для анализа химических структур:

- `rdkit`: для обработки SMILES и построения молекул;
- `descriptastorus`: генерация дескрипторов RDKit2D;
- `mordred`: расширенная генерация химико-информатических дескрипторов;
- `pandas`/`numpy`: для работы с табличными и числовыми данными.

Импортируются генераторы дескрипторов, включая `MakeGenerator`, с возможностью получения предобученных дескрипторных векторов.

In [40]:
import pandas as pd


from rdkit.Chem import PandasTools
from rdkit.Chem import MACCSkeys
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
from mordred import Calculator, descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

## Загрузка данных

Загружаются обучающая и тестовая выборки. В качестве индекса используется идентификатор CASRN. 

In [41]:
train = pd.read_csv('../data/row/simplified_data/train.csv', index_col = 'CASRN')
test = pd.read_csv('../data/row/simplified_data/test.csv', index_col = 'CASRN')

## Конвертация SMILES в молекулы
С помощью RDKit SMILES-строки преобразуются в молекулярные объекты (ROMol). Это необходимо для извлечения структурных признаков.

In [42]:
PandasTools.AddMoleculeColumnToFrame(train,smilesCol='SMILES')
PandasTools.AddMoleculeColumnToFrame(test,smilesCol='SMILES')

# ECFP6 Bits

#### Генерация ECFP6 с заданными параметрами (радиус 3, размер 2048 бит)

Используется объект `GetMorganGenerator` из RDKit для создания фингерпринтов ECFP6:

- `radius=3` — радиус охвата атомов (ECFP6);
- `fpSize=2048` — размер выходного бинарного вектора.



In [43]:
generator = GetMorganGenerator(radius=3, fpSize=2048)

In [44]:
train_ECFP6 = [generator.GetFingerprint(x) for x in train['ROMol']]
test_ECFP6 = [generator.GetFingerprint(x) for x in test['ROMol']]

#### Преобразование ECFP6-фингерпринтов в списки

Преобразуются RDKit-фингерпринты в обычные списки Python (поэлементно: 0/1).  
Это необходимо для последующей упаковки в датафреймы или массивы `NumPy`, чтобы использовать их в качестве входных данных в модели машинного обучения.

In [45]:
train_ecfp6_lists = [list(l) for l in train_ECFP6]
test_ecfp6_lists = [list(l) for l in test_ECFP6]

Генерируем список строк `Bit_0`, `Bit_1`, ..., `Bit_2047`.


In [46]:
ecfp6_name = [f'Bit_{i}' for i in range(2048)]

Создаются DataFrame для тренировочных и тестовых данных, где строки индексированы по индексам `train` и `test`, а столбцы названы в соответствии с `ecfp6_name`.


In [47]:
train_ecfp6_df = pd.DataFrame(train_ecfp6_lists, index = train.index, columns=ecfp6_name)
test_ecfp6_df = pd.DataFrame(test_ecfp6_lists, index = test.index, columns=ecfp6_name)

Вывод размеров DataFrame для тренировочных и тестовых данных: `train_ecfp6_df` и `test_ecfp6_df`.


In [48]:
train_ecfp6_df.shape, test_ecfp6_df.shape


((8221, 2048), (2849, 2048))

Сохранение DataFrame с тренировочными и тестовыми данными в CSV файлы: `train_ecfp6_bits.csv` и `test_ecfp6_bits.csv`.


In [49]:
train_ecfp6_df.to_csv('../data/descriptors/train_ecfp6_bits.csv')
test_ecfp6_df.to_csv('../data/descriptors/test_ecfp6_bits.csv')

#### Далее аналогично

## ECFP6 counts

Создание генератора с подсчётом частот (counts), радиус = 3, размер вектора = 2048

In [50]:
generator = GetMorganGenerator(radius=3, fpSize=2048, countSimulation=True)

Генерация фингерпринтов для train и test

In [51]:
train_ECFP6_counts = [generator.GetFingerprint(x) for x in train['ROMol']]
test_ECFP6_counts = [generator.GetFingerprint(x) for x in test['ROMol']]

Преобразование объектов типа RDKit ExplicitBitVect в списки

In [52]:
train_ecfp6_counts_lists = [list(l) for l in train_ECFP6_counts]
test_ecfp6__counts_lists = [list(l) for l in test_ECFP6_counts]

Создание DataFrame

In [53]:
train_ecfp6_counts_df = pd.DataFrame(train_ecfp6_counts_lists, index = train.index, columns=ecfp6_name)
test_ecfp6_counts_df = pd.DataFrame(test_ecfp6__counts_lists, index = test.index, columns=ecfp6_name)

In [54]:
train_ecfp6_counts_df.shape, test_ecfp6_counts_df.shape

((8221, 2048), (2849, 2048))

In [55]:
train_ecfp6_counts_df.to_csv('../data/descriptors/train_ecfp6_counts.csv')
test_ecfp6_counts_df.to_csv('../data/descriptors/test_ecfp6_counts.csv')

## MACCS keys

#### Генерация MACCS ключей для молекул

Генерация MACCS fingerprints для обучающей и тестовой выборок

In [56]:
train_maccs = [MACCSkeys.GenMACCSKeys(x) for x in train['ROMol']]
test_maccs = [MACCSkeys.GenMACCSKeys(x) for x in test['ROMol']]

 Преобразуем RDKit объекты фингерпринтов в списки битов (0 или 1)

In [57]:
train_maccs_lists = [list(l) for l in train_maccs]
test_maccs_lists = [list(l) for l in test_maccs]

Генерируем имена колонок: MACCS включает 167 битов (от 0 до 166)

In [58]:
maccs_name = [f'Bit_{i}' for i in range(167)]

Создаём датафреймы с индексами как в оригинальных данных и названиями колонок

In [59]:
train_maccs_df = pd.DataFrame(train_maccs_lists, index = train.index, columns=maccs_name)
test_maccs_df = pd.DataFrame(test_maccs_lists, index = test.index, columns=maccs_name)

Проверка размеров полученных датафреймов

In [60]:
train_maccs_df.shape, test_maccs_df.shape


((8221, 167), (2849, 167))

Сохранение MACCS фингерпринтов в CSV

In [61]:
train_maccs_df.to_csv('../data/descriptors/train_maccs.csv')
test_maccs_df.to_csv('../data/descriptors/test_maccs.csv')

## RDKit

Инициализация генератора дескрипторов RDKit2D

In [62]:
generator = MakeGenerator(("RDKit2D",))

Генерация RDKit2D дескрипторов с помощью MakeGenerator

In [63]:
train_rdkit2d = [generator.process(x)[1:] for x in train['SMILES']]
test_rdkit2d = [generator.process(x)[1:] for x in test['SMILES']]

Получаем имена всех дескрипторов (начиная со второго — пропускаем ID)

In [64]:
rdkit2d_name = []
for name, numpy_type in generator.GetColumns():
    rdkit2d_name.append(name)

Создание датафреймов RDKit2D дескрипторов

In [65]:
train_rdkit2d_df = pd.DataFrame(train_rdkit2d, index = train.index, columns=rdkit2d_name[1:])
test_rdkit2d_df = pd.DataFrame(test_rdkit2d, index = test.index, columns=rdkit2d_name[1:])

Проверка размеров датафреймов

In [66]:
train_rdkit2d_df.shape, test_rdkit2d_df.shape

((8221, 200), (2849, 200))

Сохранение RDKit2D дескрипторов в .csv

In [67]:
train_rdkit2d_df.to_csv('../data/descriptors/train_rdkit2d.csv')
test_rdkit2d_df.to_csv('../data/descriptors/test_rdkit2d.csv')

## Mordred

Создаем объект калькулятора молекулярных дескрипторов Mordred, игнорируя 3D-данные (так как они не нужны без SDF или MOL файла)

In [68]:
mordred_calc = Calculator(descriptors, ignore_3D=True) 

Применяем калькулятор к наборам молекул для обучающей и тестовой выборок и сохраняем результат в DataFrame

In [69]:
train_mordred = mordred_calc.pandas([mol for mol in train['ROMol']])
test_mordred = mordred_calc.pandas([mol for mol in test['ROMol']])

  2%|▏         | 125/8221 [00:10<09:38, 14.00it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 178/8221 [00:13<06:25, 20.88it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|▎         | 231/8221 [00:16<06:19, 21.04it/s]

...

 67%|██████▋   | 1917/2849 [02:04<01:18, 11.89it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 72%|███████▏  | 2041/2849 [02:10<00:37, 21.75it/s]

c:\Users\PC\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 2849/2849 [02:50<00:00, 16.68it/s]


Проверка размеров датафреймов

In [70]:
train_mordred.shape, test_mordred.shape

((8221, 1613), (2849, 1613))

Удаляем ненумерические признаки (оставляем только числовые типы данных)

In [71]:
train_mordred = train_mordred.select_dtypes(include=['float64', 'int64', 'float'])

Приводим тестовый набор данных к тем же признакам, что и в обучающем наборе

In [72]:
test_mordred = test_mordred[list(train_mordred)]

Выводим размеры обучающего и тестового наборов данных

In [73]:
train_mordred.shape, test_mordred.shape

((8221, 1056), (2849, 1056))

Выводим первые строки тестового набора данных для проверки его структуры

In [74]:
test_mordred.head(1)

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,0,0,38.352025,2.405459,4.633113,38.352025,1.237162,4.31797,3.489499,0.112564,...,6.985642,9.897721,79.938464,432.287574,6.088557,3531,39,146.0,161.0,7.083333


Сравниваем список признаков в обучающем и тестовом наборах данных, чтобы убедиться, что они одинаковы

In [75]:
list(train_mordred) == list(test_mordred)

True

Устанавливаем индексы для обучающего и тестового наборов данных, чтобы они совпадали с исходными индексами

In [76]:
train_mordred.index = train.index
test_mordred.index = test.index

Выводим размеры обучающего и тестового наборов данных

In [77]:
train_mordred.shape, test_mordred.shape


((8221, 1056), (2849, 1056))

Сохраняем обучающий и тестовый наборы данных с дескрипторами в CSV файлы для дальнейшего использования

In [78]:
train_mordred.to_csv('../data/descriptors/train_mordred.csv')
test_mordred.to_csv('../data/descriptors/test_mordred.csv')